<a href="https://colab.research.google.com/github/moridin04/CCMACLRL_EXERCISES_COM221ML/blob/main/Exercise9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 9: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all classification models

Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e10/overview



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

## Dataset File

In [ ]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/train.csv?raw=true'
df = pd.read_csv(dataset_url)

## Test File

In [ ]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [ ]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


## Sample Submission File

In [ ]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [ ]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           39098 non-null  int64  
 1   loan_status  39098 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 611.0 KB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [ ]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [ ]:
df['loan_status'].value_counts()

,count
loan_status,
0,50295
1,8350


In [ ]:
df['person_home_ownership'] = df['person_home_ownership'].map({"RENT" : 1 , "OWN" : 0})
df['loan_intent'] = df['loan_intent'].map({"PERSONAL" : 1 , "MORTGAGE" : 2 , "MEDICAL" : 3, "VENTURE" : 4, "EDUCATION" : 5})
df['loan_grade'] = df['loan_grade'].map({"A" : 1 , "B" : 2 , "C" : 3, "D" : 4, "E" : 5})
df['cb_person_default_on_file'] = df['cb_person_default_on_file'].map({"Y" : 1 , "N" : 0})

In [ ]:
df.isnull().sum()

,0
id,0
person_age,0
person_income,0
person_home_ownership,24913
person_emp_length,0
loan_intent,15413
loan_grade,182
loan_amnt,0
loan_int_rate,0
loan_percent_income,0


In [ ]:
columns_with_null = ['person_home_ownership', 'loan_intent', 'loan_grade']
for column in columns_with_null:
    df['person_home_ownership'] = df['person_home_ownership'].fillna(df['person_home_ownership'].mode()[0])
    df['loan_intent'] = df['loan_intent'].fillna(df['loan_intent'].mode()[0])
    df['loan_grade'] = df['loan_grade'].fillna(df['loan_grade'].mode()[0])

In [ ]:
df.isnull().sum()

,0
id,0
person_age,0
person_income,0
person_home_ownership,0
person_emp_length,0
loan_intent,0
loan_grade,0
loan_amnt,0
loan_int_rate,0
loan_percent_income,0


In [ ]:
x = df.drop(columns = ['id', 'loan_status']).values
y = df['loan_status'].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

## 1. Train a KNN Classifier

In [ ]:
score_list = {}

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 13)
knn.fit(x_train, y_train)
knn.score(x_test, y_test)

0.8954757303626236

- Perform cross validation

In [ ]:
knn_scores = cross_val_score(knn, x, y, cv = 5)
knn_scores

array([0.89419388, 0.89317077, 0.8953875 , 0.89351181, 0.89530224])

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (knn_scores.mean(), knn_scores.std()))
score_list["KNN Classifier"] = knn_scores.mean()

0.89 accuracy with a standard deviation of 0.00


## 2. Train a Logistic Regression Classifier

In [ ]:
LR = LogisticRegression()
LR.fit(x_train, y_train)
LR_score = LR.score(x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- Perform cross validation

In [ ]:
LR_scores = cross_val_score(LR, x, y, cv = 5)
LR_scores

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

array([0.8817461 , 0.88072299, 0.87978515, 0.88063774, 0.88191662])

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (LR_scores.mean(), LR_scores.std()))
score_list["Logistic Regression Classifier"] = LR_scores.mean()

0.88 accuracy with a standard deviation of 0.00


## 3. Train a Naive Bayes Classifier

In [ ]:
NBC = GaussianNB()
NBC.fit(x_train, y_train)
NBC.score(x_test, y_test)

0.8827441173127203

- Perform cross validation

In [ ]:
NBC_scores = cross_val_score(NBC, x, y, cv = 5)
NBC_scores

array([0.87893256, 0.88345127, 0.88191662, 0.88507119, 0.88336602])

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (NBC_scores.mean(), NBC_scores.std()))
score_list["Naive Bayes Classifier"] = NBC_scores.mean()

0.88 accuracy with a standard deviation of 0.00


## 4. Train a SVM Classifier

In [ ]:
SVC = SVC()
SVC.fit(x_train, y_train)
SVC.score(x_test, y_test)

0.8578492667954984

- Perform cross validation

In [229]:
SVC_scores = cross_val_score(SVC, x, y, cv = 5)
SVC_scores

array([0.85761787, 0.85761787, 0.85761787, 0.85753261, 0.85761787])

In [230]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (SVC_scores.mean(), SVC_scores.std()))
score_list["SVM Classifier"] = SVC_scores.mean()

0.86 accuracy with a standard deviation of 0.00


## 5. Train a Decision Tree Classifier

In [231]:
DTC = DecisionTreeClassifier(random_state = 1)
DTC.fit(x_train, y_train)
DTC_score = DTC.score(x_test, y_test)

- Perform cross validation

In [232]:
DTC_scores = cross_val_score(DTC, x, y, cv = 5)
DTC_scores

array([0.885668  , 0.88788473, 0.88745844, 0.88447438, 0.88404809])

In [233]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (DTC_scores.mean(), DTC_scores.std()))
score_list["Decision Tree Classifier"] = DTC_scores.mean()

0.89 accuracy with a standard deviation of 0.00


## 6. Train a Random Forest Classifier

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [234]:
RFC = RandomForestClassifier(criterion = 'gini')
RFC.fit(x_train_scaled, y_train)
RFC_score = RFC.score(x_test, y_test)
score_list["Random Forest Classifier"] = RFC_score
print(f"Score is {RFC_score}")

Score is 0.928782539502103


## 7. Compare all the performance of all classification models

In [235]:
score_list = list(score_list.items())
for alg, score in score_list:
    print(f"{alg} Score is {str(score)[:4]} ")

KNN Classifier Score is 0.89 
Logistic Regression Classifier Score is 0.88 
Naive Bayes Classifier Score is 0.88 
SVM Classifier Score is 0.85 
Decision Tree Classifier Score is 0.88 
Random Forest Classifier Score is 0.92 


The best algorithm is **Random Forest Classifier**.

## 9. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [236]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'loan_status': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

NameError: name 'model' is not defined